# 이소오리엔틴 회귀모델 LightGBM

## 전처리 데이터 불러오기

In [ ]:
#실험군 스케일링 데이터 불러오기
import pandas as pd

df_exp_scaled = pd.read_csv('df_exp_scale.csv')

In [ ]:
df_exp_scaled

## 독립변수, 목표변수 데이터 할당

In [ ]:
X=df_exp_scaled.iloc[:, 1:5]

In [ ]:
X.shape

In [ ]:
Y=df_exp_scaled['ingredient'].values

## train,test set split

In [ ]:
from sklearn.model_selection import train_test_split                            #Test, Train 세트 라이브러리 불러오기

#테스트,트레인 세트 데이터 셋 분할 및 할당
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.3, 
                                                    random_state=50)

## RandomForest 실험군 모델링

In [ ]:
df_exp_scaled.columns

In [ ]:
X_train.shape

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

xgb = XGBRegressor()
xgb.fit(X_train, Y_train)
r2_score(Y_test, xgb.predict(X_test))

In [ ]:
%%time
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# 여기에 그리드 탐색 코드를 포함하세요.

In [ ]:
grid.best_params_

In [ ]:
grid_y_pred = grid.predict(X_test)

In [ ]:
print(grid_y_pred)

### XGBoost 회귀의 RMSE, R^2 값

In [ ]:
import math
from sklearn.metrics import mean_squared_error

grid_rmse = math.sqrt(mean_squared_error(Y_test, grid_y_pred))
print("RMSE:%f" %(grid_rmse))

In [ ]:
from sklearn.metrics import r2_score

grid_r2 = r2_score(Y_test, grid_y_pred)
print('XGBoost R2 score with parameter tunning: ', grid_r2)